In [1]:
import pandas as pd
import numpy as np
import sqlite3 as sql
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In this Notebook I will practice building and organizing SQL queries. This file is a continuation of chinook.ipynb. To do this I wil, connect to [Chinook Database](https://github.com/lerocha/chinook-database) using SQLite3. This is sample database that represent a digital media store. Database model is shown below.

![chinook_schema](https://user-images.githubusercontent.com/51002300/71028739-2e674500-210e-11ea-86c4-eff1258c738f.JPG)

Link to [SQL Style Guide](https://www.sqlstyle.guide/)

Connection to database

In [2]:
connection = sql.connect('chinook.db')

In [3]:
query = '''SELECT * FROM sqlite_master'''
pd.read_sql_query(query, connection)

,type,name,tbl_name,rootpage,sql
0,table,album,album,2,CREATE TABLE [album]\n(\n [album_id] INTEGE...
1,table,artist,artist,3,CREATE TABLE [artist]\n(\n [artist_id] INTE...
2,table,customer,customer,4,CREATE TABLE [customer]\n(\n [customer_id] ...
3,table,employee,employee,5,CREATE TABLE [employee]\n(\n [employee_id] ...
4,table,genre,genre,6,CREATE TABLE [genre]\n(\n [genre_id] INTEGE...
5,table,invoice,invoice,7,CREATE TABLE [invoice]\n(\n [invoice_id] IN...
6,table,invoice_line,invoice_line,8,CREATE TABLE [invoice_line]\n(\n [invoice_l...
7,table,media_type,media_type,9,CREATE TABLE [media_type]\n(\n [media_type_...
8,table,playlist,playlist,10,CREATE TABLE [playlist]\n(\n [playlist_id] ...
9,table,playlist_track,playlist_track,11,CREATE TABLE [playlist_track]\n(\n [playlis...


#### Practising clear writing of queries

In [4]:
query = '''select ta.artist_name artist, count(*) tracks_sold from invoice_line il
inner join (select t.track_id, ar.name artist_name from track t
inner join album al on al.album_id = t.album_id
inner join artist ar on ar.artist_id = al.artist_id) ta
on ta.track_id = il.track_id group by 1 order by 2 desc limit 10;'''
pd.read_sql_query(query, connection)

,artist,tracks_sold
0,Queen,192
1,Jimi Hendrix,187
2,Nirvana,130
3,Red Hot Chili Peppers,130
4,Pearl Jam,129
5,AC/DC,124
6,Guns N' Roses,124
7,Foo Fighters,121
8,The Rolling Stones,117
9,Metallica,106


In [5]:
query = '''
SELECT 
        ta.artist_name artist, 
        count(*) tracks_sold
FROM invoice_line il
INNER JOIN (
            SELECT 
                t.track_id, 
                ar.name artist_name 
            FROM track t
            INNER JOIN album al on al.album_id = t.album_id
            INNER JOIN artist ar on ar.artist_id = al.artist_id
            ) ta
            ON ta.track_id = il.track_id 
GROUP BY 1 
ORDER BY 2 DESC 
LIMIT 10
'''
pd.read_sql_query(query, connection)

,artist,tracks_sold
0,Queen,192
1,Jimi Hendrix,187
2,Nirvana,130
3,Red Hot Chili Peppers,130
4,Pearl Jam,129
5,AC/DC,124
6,Guns N' Roses,124
7,Foo Fighters,121
8,The Rolling Stones,117
9,Metallica,106


#### WITH clause

##### Info about the tracks from a single album

In [6]:
query = '''
SELECT * FROM
    (
     SELECT
         t.name,
         ar.name artist,
         al.title album_name,
         mt.name media_type,
         g.name genre,
         t.milliseconds length_milliseconds
     FROM track t
     INNER JOIN media_type mt ON mt.media_type_id = t.media_type_id
     INNER JOIN genre g ON g.genre_id = t.genre_id
     INNER JOIN album al ON al.album_id = t.album_id
     INNER JOIN artist ar ON ar.artist_id = al.artist_id
    )
WHERE album_name = "Jagged Little Pill"
'''
pd.read_sql_query(query, connection).head(3)

,name,artist,album_name,media_type,genre,length_milliseconds
0,All I Really Want,Alanis Morissette,Jagged Little Pill,MPEG audio file,Rock,284891
1,You Oughta Know,Alanis Morissette,Jagged Little Pill,MPEG audio file,Rock,249234
2,Perfect,Alanis Morissette,Jagged Little Pill,MPEG audio file,Rock,188133


In [7]:
query = '''
WITH track_info AS
    (                
     SELECT
         t.name,
         ar.name artist,
         al.title album_name,
         mt.name media_type,
         g.name genre,
         t.milliseconds length_milliseconds
     FROM track t
     INNER JOIN media_type mt ON mt.media_type_id = t.media_type_id
     INNER JOIN genre g ON g.genre_id = t.genre_id
     INNER JOIN album al ON al.album_id = t.album_id
     INNER JOIN artist ar ON ar.artist_id = al.artist_id
    )

SELECT * FROM track_info
WHERE album_name = "Jagged Little Pill"
'''
pd.read_sql_query(query, connection).head(3)

,name,artist,album_name,media_type,genre,length_milliseconds
0,All I Really Want,Alanis Morissette,Jagged Little Pill,MPEG audio file,Rock,284891
1,You Oughta Know,Alanis Morissette,Jagged Little Pill,MPEG audio file,Rock,249234
2,Perfect,Alanis Morissette,Jagged Little Pill,MPEG audio file,Rock,188133


#### Summary data for every playlist in the Chinook dataset

In [8]:
query = '''
WITH playlist_summary AS
    (
    SELECT
        p.playlist_id,
        p.name playlist_name,
        t.name track_name,
        (t.milliseconds/1000) length_seconds
    FROM playlist p
    LEFT JOIN playlist_track pt ON pt.playlist_id = p.playlist_id
    LEFT JOIN track t ON t.track_id = pt.track_id
    )
SELECT
    playlist_id,
    playlist_name,
    COUNT(track_name) number_of_tracks,
    SUM(length_seconds) length_seconds
FROM playlist_summary
GROUP BY 1,2
ORDER BY 1 ASC
'''
pd.read_sql_query(query, connection)

,playlist_id,playlist_name,number_of_tracks,length_seconds
0,1,Music,3290,876049.0
1,2,Movies,0,NaN
2,3,TV Shows,213,500987.0
3,4,Audiobooks,0,NaN
4,5,90’s Music,1477,397970.0
5,6,Audiobooks,0,NaN
6,7,Movies,0,NaN
7,8,Music,3290,876049.0
8,9,Music Videos,1,294.0
9,10,TV Shows,213,500987.0


#### Creating a view

In [9]:
query = '''
SELECT sql FROM sqlite_master WHERE type = 'view'
'''
pd.read_sql_query(query, connection)    

,sql
0,CREATE VIEW view_name AS\n SELECT * FROM track
1,CREATE VIEW customer_gt_90_dollars AS\n SEL...
2,CREATE VIEW customer_usa AS \n SELECT * FR...


In [ ]:
query = '''
CREATE VIEW customer_usa AS 
     SELECT * FROM customer
     WHERE country = "USA"
'''
pd.read_sql_query(query, connection)       

In [ ]:
query = '''
CREATE VIEW customer_gt_90_dollars AS
    SELECT c.* FROM invoice i
    INNER JOIN customer c ON c.customer_id = i.customer_id
    GROUP BY 1
    HAVING SUM(i.total) > 90
'''
pd.read_sql_query(query, connection)    

In [11]:
query = '''
SELECT * FROM customer_gt_90_dollars   
'''
pd.read_sql_query(query, connection).head()

,customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
1,3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
2,5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4
3,6,Helena,Holý,None,Rilská 3174/6,Prague,None,Czech Republic,14300,+420 2 4177 0449,None,hholy@gmail.com,5
4,13,Fernanda,Ramos,None,Qe 7 Bloco G,Brasília,DF,Brazil,71020-677,+55 (61) 3363-5547,+55 (61) 3363-7855,fernadaramos4@uol.com.br,4


#### Customers in the USA or have spent more than $90

In [12]:
query = '''
SELECT * FROM customer_usa
UNION
SELECT * FROM customer_gt_90_dollars
'''
pd.read_sql_query(query, connection).sum()

customer_id                                                     721
first_name        LuísFrançoisFrantišekHelenaFernandaFrankJackMi...
last_name         GonçalvesTremblayWichterlováHolýRamosHarrisSmi...
address           Av. Brigadeiro Faria Lima, 21701498 rue Bélang...
city              São José dos CamposMontréalPraguePragueBrasíli...
country           BrazilCanadaCzech RepublicCzech RepublicBrazil...
phone             +55 (12) 3923-5555+1 (514) 721-4711+420 2 4172...
email             luisg@embraer.com.brftremblay@gmail.comfrantis...
support_rep_id                                                  104
dtype: object

#### Customers who are in the USA and have spent more than $90

In [13]:
query = '''
SELECT * FROM customer_usa
INTERSECT
SELECT * FROM customer_gt_90_dollars
'''
pd.read_sql_query(query, connection).sum()

customer_id                                                      80
first_name                                      JackDanKathyHeather
last_name                                   SmithMillerChaseLeacock
address           1 Microsoft Way541 Del Medio Avenue801 W 4th S...
city                                RedmondMountain ViewRenoOrlando
state                                                      WACANVFL
country                                                USAUSAUSAUSA
postal_code                           98052-830094040-1118950332801
phone             +1 (425) 882-8080+1 (650) 644-3358+1 (775) 223...
email             jacksmith@microsoft.comdmiller@comcast.comkach...
support_rep_id                                                   18
dtype: object

#### Customers who are in the USA and have not spent $90

In [14]:
query = '''
SELECT * FROM customer_usa
EXCEPT
SELECT * FROM customer_gt_90_dollars
'''
pd.read_sql_query(query, connection).sum()

customer_id                                                     206
first_name        FrankMichelleTimJohnFrankVictorRichardPatrickJ...
last_name         HarrisBrooksGoyerGordonRalstonStevensCunningha...
address           1600 Amphitheatre Parkway627 Broadway1 Infinit...
city              Mountain ViewNew YorkCupertinoBostonChicagoMad...
state                                            CANYCAMAILWITXAZUT
country                                 USAUSAUSAUSAUSAUSAUSAUSAUSA
postal_code       94043-135110012-261295014211360611537037611085...
phone             +1 (650) 253-0000+1 (212) 221-3546+1 (408) 996...
email             fharris@google.commichelleb@aol.comtgoyer@appl...
support_rep_id                                                   35
dtype: object

#### Customers that are in the USA and have purchased more than $90 are assigned to each sales support agent

In [15]:
query = '''
WITH customers_usa_gt_90 AS
    (
     SELECT * FROM customer_usa
     INTERSECT
     SELECT * FROM customer_gt_90_dollars
    )
SELECT 
    e.first_name || " " || e.last_name employee_name,
    COUNT(DISTINCT c.customer_id) customers_usa_gt_90 
FROM 
    employee e 
LEFT JOIN customers_usa_gt_90 c ON e.employee_id = c.support_rep_id

WHERE e.title = 'Sales Support Agent'
GROUP BY 1
ORDER BY 1
'''
pd.read_sql_query(query, connection)

,employee_name,customers_usa_gt_90
0,Jane Peacock,0
1,Margaret Park,2
2,Steve Johnson,2


#### Three named subqueries that build on each other

In [16]:
query = '''
WITH
    usa AS
        (
        SELECT * FROM customer
        WHERE country = 'USA'
        ),
    last_name_g AS
        (
         SELECT * FROM usa
         WHERE last_name LIKE 'G%'
        ),
    state_ca AS
        (
        SELECT * FROM last_name_g
        WHERE state = 'CA'
        )        
SELECT
    first_name,
    last_name,
    country,
    state
FROM state_ca        
'''
pd.read_sql_query(query, connection)

,first_name,last_name,country,state
0,Tim,Goyer,USA,CA


#### Multiple named subqueries in a WITH clause to gather total sales data on customers from India

In [17]:
query = '''
WITH 
    customers_india AS
    (
    SELECT * FROM customer c
    WHERE country = 'India'
    ),
    total_purchases AS
    (
    SELECT 
        customer_id,
        SUM(total) total
    FROM invoice
    GROUP BY customer_id
    )
    
SELECT 
    ci.first_name || " " || ci.last_name AS customer_name,
    tp.total
FROM customers_india ci
LEFT JOIN total_purchases tp ON tp.customer_id = ci.customer_id
ORDER BY customer_name ASC 
'''
pd.read_sql_query(query, connection)

,customer_name,total
0,Manoj Pareek,111.87
1,Puja Srivastava,71.28
